In [1]:
#相关库引用
import requests
import urllib
import urllib.request
import json
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import matplotlib.pyplot as plt

import random
def get_ip_list():
    print("正在获取代理列表...")
    headers = {'User-Agent':
                   'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
    url = 'http://www.xicidaili.com/nn/'
    html = requests.get(url=url, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    ips = soup.find(id='ip_list').find_all('tr')
    ip_list = []
    for i in range(1, len(ips)):
        ip_info = ips[i]
        tds = ip_info.find_all('td')
        ip_list.append(tds[1].text + ':' + tds[2].text)
    print("代理列表抓取成功.")
    return ip_list
ip_list = get_ip_list()   

def get_random_ip():
    print("正在设置随机代理...")
    proxy_list = []
    for ip in ip_list:
        proxy_list.append('http://' + ip)
    proxy_ip = random.choice(proxy_list)
    proxies = {'http': proxy_ip}
    print("代理设置成功.")
    return proxies



In [2]:
#请求并获取url链接资源
def get_html_text(url):
    headers = {'User-Agent':
                   'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
#     proxies = get_random_ip()
#     r = requests.get(url, headers=headers, proxies=proxies)
    r = requests.get(url, headers=headers)
    try:
        r.raise_for_status()
    except Exception as e:
        print(e)
        return None
    return r.text

#从服务器返回的资源中筛选国内热门城市的信息
def get_cityid_name(url):
    soup = get_html_text(url)
    soup = BeautifulSoup(soup, 'lxml')
    items = soup.find_all('li', class_="item")
    info = []
    for item in items:
        city = {}
        city['city_id'] = item.find('a').attrs['data-id']
        city['city_name'] = item.find('div', class_='title').text.split()[0]
        city['nums'] = int(item.find('b').text)
        city['describe'] = item.find('div', class_='detail').text.split()
        city['TOP3'] = item.find('dd').text.split()[1:]
#         nums += city['nums']
        info.append(city)
    return info
# get_cityid_name('http://www.mafengwo.cn/mdd/citylist/21536.html?mddid=21536&page=90')

def get_all_cityid(page):
    all = []
#     page = 200
    for page_id in range(1, page):
        url = 'http://www.mafengwo.cn/mdd/citylist/21536.html?mddid=21536&page={}'.format(page_id)
        time.sleep(2)
#         print(url)
#         print(get_cityid_name(url))
        all.extend(get_cityid_name(url))
    return all
# len(get_all_cityid(3))


def to_dataframe(li):
    data = pd.DataFrame(li)
    order = ['city_id', 'city_name', 'nums', 'TOP3', 'describe']
    data = data[order]
    return data

# city_data = to_dataframe(get_all_cityid(422))


In [33]:
# data.to_csv('city_data.csv')
#保存城市id等数据
city_data.to_csv("city_data.csv",encoding="utf_8_sig")

In [89]:
#获取每个城市游记的编号
def get_yj_dataiid(url):
    soup = get_html_text(url)
    soup = BeautifulSoup(soup, 'lxml')
    items = soup.find_all('li', class_="post-item clearfix")
    info = []
    for item in items:
        iid = item.find('a', class_="btn-ding").attrs['data-iid']
        info.append(iid)
    return info

def get_yj_infor(url):
    soup = get_html_text(url)
    soup = BeautifulSoup(soup, 'lxml')
    info = {}
    items = ['time','day' ,'people' ,'cost' ,]
    for item in items:
        try:
            info[item] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_= item).text
        except:
            info[item] = 'None'
    
    note_yj = soup.find_all('p', class_='_j_note_content _j_seqitem')
    note_text = []
    for i in note_yj:
        note_text.append(''.join(i.text.split()))
    info['note_text'] = note_text
#     info['time'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='time').text
#     info['day'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='day').text
#     info['people'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='people')
#     info['cost'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='cost')
    return info


def to_flip_over(city_id, page):
    id_list = []
    url = 'http://www.mafengwo.cn/yj/' + str(city_id) + '/1-0-{}.html'
    for page_id in range(1,page):
        url = url.format(page_id)
        time.sleep(1)
#         time.sleep(2)
        id_list.extend(get_yj_dataiid(url))
        
    return id_list


#稻城{id：10061}为例
youji_id = to_flip_over(10061,199)

# def get_city_all_yj_info(page):
#     youji_id = to_flip_over(10061, page)
#     all_info = []
#     for i in youji_id:
#         url = 'http://www.mafengwo.cn/i/{}.html'.format(i)
#         time.sleep(1)
#         all_info.append(get_yj_infor(url))
#     all_info = pd.DataFrame(all_info)['time', 'day', 'people', 'cost', 'note_text']
#     return all_info

# daochen_data = get_city_all_yj_info(199)
# daochen_data.to_csv("city_data.csv",encoding="utf_8_sig")        
# get_yj_dataiid('http://www.mafengwo.cn/yj/10189/1-0-0.html')


In [3]:
# len(youji_id)
def get_city_all_yj_info():
#     youji_id = to_flip_over(10061, page)
    all_info = []
    for i in youji_id:
        url = 'http://www.mafengwo.cn/i/{}.html'.format(i)
        time.sleep(3)
        all_info.append(get_yj_infor(url))
        dcdt.append(get_yj_infor(url))
    all_info = pd.DataFrame(all_info)['time', 'day', 'people', 'cost', 'note_text']
    return all_info
# dcdt = []
# daochen_data = get_city_all_yj_info()
# daochen_data.to_csv("city_data.csv",encoding="utf_8_sig")        
# get_yj_dataiid('http://www.mafengwo.cn/yj/10189/1-0-0.html')

In [116]:
dcdt
yj_d = pd.DataFrame(dcdt)
yj_d.to_csv("yj_d1.csv",encoding="utf_8_sig")

In [95]:
import random
def get_ip_list():
    print("正在获取代理列表...")
    headers = {'User-Agent':
                   'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
    url = 'http://www.xicidaili.com/nn/'
    html = requests.get(url=url, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    ips = soup.find(id='ip_list').find_all('tr')
    ip_list = []
    for i in range(1, len(ips)):
        ip_info = ips[i]
        tds = ip_info.find_all('td')
        ip_list.append(tds[1].text + ':' + tds[2].text)
    print("代理列表抓取成功.")
    return ip_list
ip_list = get_ip_list()   

def get_random_ip():
    print("正在设置随机代理...")
    proxy_list = []
    for ip in ip_list:
        proxy_list.append('http://' + ip)
    proxy_ip = random.choice(proxy_list)
    proxies = {'http': proxy_ip}
    print("代理设置成功.")
    return proxies

# get_random_ip()
resp = requests.get(key.uri, headers = self.headers, proxies=proxies)


正在获取代理列表...
代理列表抓取成功.
正在设置随机代理...
代理设置成功.


{'http': 'http://49.71.66.82:8118'}

In [28]:
# BeautifulSoup(get_html_text('http://www.mafengwo.cn/i/10074422.html'), 'lxml').find('div', class_='tarvel_dir_list clearfix')
def get_yj_infor(url):
    soup = get_html_text(url)
    soup = BeautifulSoup(soup, 'lxml')
    info = {}
    items = ['time','day' ,'people' ,'cost' ,]
    for item in items:
        try:
            info[item] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_= item).text
        except:
            info[item] = 'None'
    
    note_yj = soup.find_all('p', class_='_j_note_content _j_seqitem')
    note_text = []
    for i in note_yj:
        note_text.append(''.join(i.text.split()))
    info['note_text'] = note_text
#     info['time'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='time').text
#     info['day'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='day').text
#     info['people'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='people')
#     info['cost'] = soup.find('div', class_='tarvel_dir_list clearfix').find('li', class_='cost')
    return info
get_yj_infor('http://www.mafengwo.cn/i/10074422.html')

{'cost': '人均费用/5000RMB',
 'day': '出行天数/9 天',
 'note_text': ['2016年十月，一部《从你的全世界路过》让我对一个地方心生向往。那里有磅礴的雪山，广阔的草原，纯净的湖泊，苍翠的原始森林。它的名字就是稻城亚丁。于是在2017年7月，在提前几个月做好了攻略，订好了飞往成都的机票以后，我和逗逼兵俩人就开启了向往已久的旅程。这里插句话，正式介绍下逗逼兵，我大学时候的恋人，现在是老公。对了，我俩都是无辣不欢的吃货，所以此行必须得从舌尖上的成都开始。',
  'Day1舌尖上的成都打卡第一天',
  '如果说重庆的代名词是辣和火锅，那么成都的代名词就是麻与辣，和数不尽的火锅、串串、冒菜、川菜、兔头、鸭脚……说起来都饿了。下飞机，到成都市区已是中午饭点了。我们在成都的第一顿就是火锅。宽窄巷子里面随便找的一家店。味道还不错，主要是食材和调料够味，辣椒也够味。',
  '由于我在飞猪上预定的酒店就在宽窄巷子，所以成都行第一站就从宽窄巷子开始。这里要强烈推荐成都的住宿，300左右的价位预定的民宿就是套房级别了，一室一厅一卫一厨一阳台的格局，可谓麻雀虽小，五脏俱全。面积两个人住的话也不算小。下图是我们入住的宽窄印象度假公寓。',
  '宽窄巷子，在举办一个马尔康摄影展，人还是非常多的。在这个摄影展里面，看到了有关川西的一些采风，内心更充满向往和期待了。',
  '从宽窄巷子出来以后，我们就去了春熙路和天府广场。这个就是比较大众化的商业广场了。哈哈😄，你要不要吃兔兔？这个怪大叔在吃兔兔吗？不，他在啃鸡爪。这家鸡爪就在春熙路步行街小吃街上，很好吃很入味，入口即化。',
  'Day2继续舌尖上的成都打卡',
  '今天的目标就是武侯祠转转，然后去和我们在网上预定的半自助旅行团，成都大脚旅行社汇合。我们预定的是11个人的半自助团，全程只有一个司机，纯玩不进店。武侯祠，这个巷子里面很多蚊子，一定要做好防蚊措施，花露水防蚊液随身带。我就是吃了这个亏，啥都没带，被蚊子咬地狂跳脚，再也没有拍照和游览的雅兴了。',
  'Day3向川西前进，汶川，四姑娘山双桥沟',
  '早早地起来，从成都出发，渐渐地离城市越来越远了，渐渐地映入眼帘的是雪山，森林，低到触手可及的蓝天和厚厚的云朵，我和逗逼兵的心情都很兴奋。在城市待久了，已经很难见到这样的原生态、

In [44]:
# url = 'http://www.mafengwo.cn/yj/10061/1-0-2.html'
# ww = 'http://www.mafengwo.cn/yj/{}/1-0-.html'
# ww.format('1')
ww = 'http://www.mafengwo.cn/yj/' + str(12345) + '/1-0-{}.html'
ww.format(321)

'http://www.mafengwo.cn/yj/12345/1-0-321.html'

In [201]:
ww=BeautifulSoup(get_html_text('http://www.mafengwo.cn/i/10074422.html'), 'lxml').find_all('', class_="post-item clearfix")[0].find('a', class_="btn-ding").attrs['data-iid']
print(ww)
int(ww)

10189346


10189346

In [186]:
soup = BeautifulSoup(aa, 'lxml')
# zz = re.compile("data-id=.*目的地")
# soup
# soup.findAll(lambda tag: len(tag.name) == 1 and not tag.attrs)
# soup.findAll(text="10794")
items = soup.find_all('li', class_="item")
# items[2].find('a').attrs['data-id']
# items[2].find('a').find('div').get_text()
# items[2].find('div', class_='detail').text
items[2].find('dd').text.split()
# items[2].find('b').text
# for item in items:
#     item.find('a').attrs['data-id']

['TOP3', '新疆国际大巴扎', '维吾尔自治区博物馆', '红山公园']

In [120]:
pd.DataFrame(youji_id).to_csv('daochen_yj_iid.csv')

In [22]:
yj_bianhao = pd.read_csv('daochen_yj_iid.csv')


In [47]:
# yj_bh = yj_bianhao.iloc[:,1]
# # yj_bh[0:10]
# getxx = []

for i in yj_bh[2200:]:
#     print(i)
    getxx.append(get_yj_infor('http://www.mafengwo.cn/i/{}.html'.format(i)))
    time.sleep(8)
    print('第{}个'.format(i))
    print(len(getxx))
    
    


第3507793个
2177
第8665122个
2178
第11215166个
2179
第7994925个
2180
第11034530个
2181
第10907570个
2182
第11214510个
2183
第8101684个
2184
第11064870个
2185
第9080121个
2186
第10063046个
2187
第9398205个
2188
第5609927个
2189
第3327099个
2190
第8239738个
2191
第3507793个
2192
第8665122个
2193
第11215166个
2194
第7994925个
2195
第11034530个
2196
第10907570个
2197
第11214510个
2198
第8101684个
2199
第11064870个
2200
第9080121个
2201
第10063046个
2202
第9398205个
2203
第5609927个
2204
第3327099个
2205
第8239738个
2206
第3507793个
2207
第8665122个
2208
第11215166个
2209
第7994925个
2210
第11034530个
2211
第10907570个
2212
第11214510个
2213
第8101684个
2214
第11064870个
2215
第9080121个
2216
第10063046个
2217
第9398205个
2218
第5609927个
2219
第3327099个
2220
第8239738个
2221
第3507793个
2222
第8665122个
2223
第11215166个
2224
第7994925个
2225
第11034530个
2226
第10907570个
2227
第11214510个
2228
第8101684个
2229
第11064870个
2230
第9080121个
2231
第10063046个
2232
第9398205个
2233
第5609927个
2234
第3327099个
2235
第8239738个
2236
第3507793个
2237
第8665122个
2238
第11215166个
2239
第7994925个
2240
第11034530个
2241

第8101684个
2709
第11064870个
2710
第9080121个
2711
第10063046个
2712
第9398205个
2713
第5609927个
2714
第3327099个
2715
第8239738个
2716
第3507793个
2717
第8665122个
2718
第11215166个
2719
第7994925个
2720
第11034530个
2721
第10907570个
2722
第11214510个
2723
第8101684个
2724
第11064870个
2725
第9080121个
2726
第10063046个
2727
第9398205个
2728
第5609927个
2729
第3327099个
2730
第8239738个
2731
第3507793个
2732
第8665122个
2733
第11215166个
2734
第7994925个
2735
第11034530个
2736
第10907570个
2737
第11214510个
2738
第8101684个
2739
第11064870个
2740
第9080121个
2741
第10063046个
2742
第9398205个
2743
第5609927个
2744
第3327099个
2745
第8239738个
2746
第3507793个
2747
第8665122个
2748
第11215166个
2749
第7994925个
2750
第11034530个
2751
第10907570个
2752
第11214510个
2753
第8101684个
2754
第11064870个
2755
第9080121个
2756
第10063046个
2757
第9398205个
2758
第5609927个
2759
第3327099个
2760
第8239738个
2761
第3507793个
2762
第8665122个
2763
第11215166个
2764
第7994925个
2765
第11034530个
2766
第10907570个
2767
第11214510个
2768
第8101684个
2769
第11064870个
2770
第9080121个
2771
第10063046个
2772
第9398205个
2773

In [48]:
len(getxx)

2946

In [49]:
pd.DataFrame(getxx).to_csv('gonglue_data.csv',encoding="utf_8_sig")

In [ ]:
import pymysql

class my_sql():
    def __init__(self,**kwargs):
        try:
            
            self.__conn = pymysql.connect(host = 'localhost',user = 'root',passwd = '',charset='utf8')
            self.__cursor = self.__conn.cursor()
            print("链接数据库")
        except:
            print("连接失败")
    
    def connects(self):
        return self.__conn
    def close(self):
        self.__conn.close()
    #查找操作
    def find(self,sql_lang):
        try:
            self.__cursor.execute(sql_lang)
            result = 
    
    
            #一、创建数据库对象connection
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    passwd = '',
    charset='utf8'
)
#二、创建数据库数据承载对象--获取游标对象cursor
cursor = conn.cursor()

#1.创建一个数据库--教务系统
#cursor.execute('create database if not exists School ')

#2.选择数据库
conn.select_db('School')

#3.删除数据库
#cursor.execute('drop database if exists School')

#4.创建数据库中的表
cursor.execute('create table if not exists Student(Sno char(10)not null,Sname char(10),Ssex char(2),Sage int,Sdept varchar(20))')

#5.删除表
#cursor.execute('drop table if exists Student')

#6.显示创建的数据库中的表
cursor.execute('select* from Student')
#得到一元祖 存放查询数据
def display():
    a=cursor.fetchone()
    print(a)


In [10]:
class my_sql():
    def __init__(self,**kwargs):
        try:
            
            self.__conn =  [1,2]
#             self.__cursor = None
            print("链接数据库")
        except:
            print("连接失败")
    
    def get(self):
        return self.__conn
ts = my_sql()
ts.get()

链接数据库


[1, 2]